# PROSJEKT INF161
## Movie recommendation project - predictions
`@author` Kristin Loka Øydna

In this file we want to create a matrix with predicted ratings for every user and movie. I was not abel to finish this.

I have imported the model from `model.ipynb`and made predictions for every user in the dataset. The problem with what I have done is that I only predict what the users would rate on movies they have already rated, so I cannot use this to replace NaN. 

In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

model = pickle.load(open("content_model.pickle", "rb"))

movieuser_df = pd.read_csv("/Users/kristinlokaoydna/UiB - Bioinformatikk/H20/INF161/Prosjekt/movieuser_df.csv")

movieuser_df

,BrukerID,FilmID,Tidsstempel,Rangering,Tittel,Action,Adventure,Animation,Children's,Comedy,...,1971-1980,1981-1990,1991-2000,Kjonn,Alder,Jobb,Postkode,Under 18-24,25-44,45-55+
0,0,791,959442983.0,2,Armageddon (1998),1,1,0,0,0,...,0,0,1,M,45.0,6.0,92103,0,0,1
1,0,2975,959443833.0,4,"Room with a View, A (1986)",0,0,0,0,0,...,0,1,0,M,45.0,6.0,92103,0,0,1
2,0,3407,959443373.0,4,"Grand Day Out, A (1992)",0,0,1,0,1,...,0,0,1,M,45.0,6.0,92103,0,0,1
3,0,189,959445005.0,4,"Far Off Place, A (1993)",0,1,0,1,0,...,0,0,1,M,45.0,6.0,92103,0,0,1
4,0,773,959443944.0,2,Anna and the King (1999),0,0,0,0,0,...,0,0,1,M,45.0,6.0,92103,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899853,2145,3490,965318885.0,2,Bronco Billy (1980),0,1,0,0,0,...,1,0,0,M,45.0,5.0,77662,0,0,1
899854,2145,240,965318885.0,3,Braddock: Missing in Action III (1988),1,0,0,0,0,...,0,1,0,M,45.0,5.0,77662,0,0,1
899855,2145,892,965319138.0,3,MacKenna's Gold (1969),0,0,0,0,0,...,0,0,0,M,45.0,5.0,77662,0,0,1
899856,2145,3807,965319221.0,5,Steel Magnolias (1989),0,0,0,0,0,...,0,1,0,M,45.0,5.0,77662,0,0,1


### Making predictions on the whole dataset from the chosen model

In [2]:
predictions = []
    
for i, row in movieuser_df.iterrows():
    X1_f = np.array(row[['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 
                       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
                       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western','1910-1940',
                       '1941-1970','1971-1980','1981-1990','1991-2000', 'Under 18-24','25-44','45-55+']]).reshape(-1, 26)

 
    pred_f = model[row.BrukerID].predict(X1_f)
    pred_f = np.clip(pred_f, 1, 5) 
    predictions.append(pred_f)
    

mse_content_f = mean_squared_error(predictions, movieuser_df['Rangering'])
rmse_content_f = np.sqrt(mse_content_f)

print("RMSE of content-based model with features on the entire dataset:",rmse_content_f)

RMSE of content-based model with features on the entire dataset: 0.9367530299158036


The RMSE is better for the entire dataset than for the test set. Hence, the model workes better when we include every user. 

In [4]:
# Creating a dataframe of the predicted ratings
df = pd.DataFrame(predictions, columns=["Pred Rang"])
df

# Adding the predicted ratings to the dataframe
merged = movieuser_df.join(df)
merged

,BrukerID,FilmID,Tidsstempel,Rangering,Tittel,Action,Adventure,Animation,Children's,Comedy,...,1981-1990,1991-2000,Kjonn,Alder,Jobb,Postkode,Under 18-24,25-44,45-55+,Pred Rang
0,0,791,959442983.0,2,Armageddon (1998),1,1,0,0,0,...,0,1,M,45.0,6.0,92103,0,0,1,2.982066
1,0,2975,959443833.0,4,"Room with a View, A (1986)",0,0,0,0,0,...,1,0,M,45.0,6.0,92103,0,0,1,3.406261
2,0,3407,959443373.0,4,"Grand Day Out, A (1992)",0,0,1,0,1,...,0,1,M,45.0,6.0,92103,0,0,1,3.326270
3,0,189,959445005.0,4,"Far Off Place, A (1993)",0,1,0,1,0,...,0,1,M,45.0,6.0,92103,0,0,1,3.511341
4,0,773,959443944.0,2,Anna and the King (1999),0,0,0,0,0,...,0,1,M,45.0,6.0,92103,0,0,1,3.221922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899853,2145,3490,965318885.0,2,Bronco Billy (1980),0,1,0,0,0,...,0,0,M,45.0,5.0,77662,0,0,1,2.000000
899854,2145,240,965318885.0,3,Braddock: Missing in Action III (1988),1,0,0,0,0,...,1,0,M,45.0,5.0,77662,0,0,1,2.333333
899855,2145,892,965319138.0,3,MacKenna's Gold (1969),0,0,0,0,0,...,0,0,M,45.0,5.0,77662,0,0,1,5.000000
899856,2145,3807,965319221.0,5,Steel Magnolias (1989),0,0,0,0,0,...,1,0,M,45.0,5.0,77662,0,0,1,5.000000


### Prediction matrix
I have created a matrix with every user id and movie id. It is filled with NaN for the movies users have not rated. We are suppose to fill in the predicted ratings here, and then save it as a csv file to hand in. And becuse of this I was not abel to create a website.  

In [5]:
movieuser_table = pd.pivot_table(movieuser_df, values="Rangering", index="FilmID", columns=['BrukerID'])
movieuser_table

BrukerID,0,1,2,3,4,5,6,7,8,9,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
FilmID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
movieuser_table.to_csv (r'/Users/kristinlokaoydna/UiB - Bioinformatikk/H20/INF161/Prosjekt/predictions.csv', index = False, header=True)